# Building Datasets For Model Training

Carrying on from [Issue #6](https://github.com/dannylyl/siamese-nn-facial-recognition/issues/6), this issue will involve steps for building the datasets for further downstream steps in this project. This would include:

1. Deciding Train Test Split Methodology and Implementation
2. Building Positive (similar) and Negative (dissimilar) pairs and labels

## 1. Imports

In [75]:
import os
import pandas as pd
import PIL
from sklearn.model_selection import train_test_split

## 2. Getting Pandas DF of Images

Similar to the notebook for issue #6, I will start by first organising the data into a pandas dataframe (personal preference I guess since I usually work with data in DFs).

In [76]:
data_dir = '../data/lfw-deepfunneled/lfw-deepfunneled'

data = []

for person in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person)
    if os.path.isdir(person_dir):
        for image in os.listdir(person_dir):
            data.append((os.path.join(person_dir, image), person))
            
df = pd.DataFrame(data, columns=['image_path', 'person'])

In [77]:
df.head()

,image_path,person
0,../data/lfw-deepfunneled/lfw-deepfunneled/Alic...,Alice_Fisher
1,../data/lfw-deepfunneled/lfw-deepfunneled/Alic...,Alice_Fisher
2,../data/lfw-deepfunneled/lfw-deepfunneled/Elle...,Ellen_Barkin
3,../data/lfw-deepfunneled/lfw-deepfunneled/Quee...,Queen_Latifah
4,../data/lfw-deepfunneled/lfw-deepfunneled/Quee...,Queen_Latifah


## 3. Train Test Split Methodology

As suggested from Issue #6, we have:

* A total of `5749` unique persons in our dataset. `1680` people have multiple images, and `4068` people with only 1 image
* A total of `13233` images, of which `9164` are part of the multiple images per person, and naturally `4068` solo images.

Thinking in a way to reduce the chances of data leakage, or just to test the model's generalisability, perhaps it would be better to perform a train test split on the individual people, and not on an image level.

If the end goal is a model that can perform one-shot facial verification on multiple people, the model should be able to perform well on many people, and so the evaluation of the model should involve assessing its performance on unseen people.

Now, something to address with that methodology, is the fact that some people have more multiple images of themselves, so performing a split on unique persons without taking into account the number of images there are per person might not be the best way to go. 

As brought up in Issue #6, we can technically have many permutations of positive and negative labels just from matching different images together, which would increase our dataset. But let's still split them while taking into account the number of images.

#### a. Creating DF with Counts, and Separating the People with Unique Counts of Images

In [78]:
person_counts = df['person'].value_counts().reset_index()
person_counts.columns = ['person', 'num_photos']

In [79]:
person_counts.head()

,person,num_photos
0,George_W_Bush,530
1,Colin_Powell,236
2,Tony_Blair,144
3,Donald_Rumsfeld,121
4,Gerhard_Schroeder,109


One good way to split our dataset into train and test sets would be to make sure the proportion of photos per person is constant across the split. This might help the model to generalise to the test set a bit better.

Firstly, let's start by checking if there are any people with unique numbers of images in the dataset. We kind of already know that there are from the cell above, which shows 5 people with non repeating `num_photos`.

In [80]:
num_photo_count= person_counts['num_photos'].value_counts()
num_photo_count.tail()

num_photos
530    1
236    1
144    1
27     1
25     1
Name: count, dtype: int64

In [81]:
unique_counts = []
for num_photos, count in num_photo_count.items():
    if count == 1:
        unique_counts.append(num_photos)
        
print(f'Number of people whose count of images is unique: {len(unique_counts)}')

Number of people whose count of images is unique: 18


#### b. Train Test Splitting the Datasets, Stratifying on Number of Photos per Person

So we have 18 people whose count of images in unique. Let's extract them out of the dataset first and do a train test split on the rest. We'll do an 80 / 20 split. I am not going to split the datasets in to Train, test and validation as this is meant to be quick, small project, and I want to keep things simple for now.

In [82]:
unique_count_people = person_counts[person_counts['num_photos'].isin(unique_counts)]
unique_count_people.tail()

,person,num_photos
21,Nestor_Kirchner,37
22,Andre_Agassi,36
23,Alvaro_Uribe,35
38,Ricardo_Lagos,27
41,Tom_Daschle,25


In [86]:
person_counts = person_counts[~person_counts['person'].isin(unique_count_people['person'])]

In [87]:
person_counts['person'].nunique()

5731

In [88]:
train_persons, test_persons = train_test_split(person_counts, test_size=0.2, stratify=person_counts['num_photos'], random_state=42)

In [89]:
train_persons['person'].nunique(), test_persons['person'].nunique()

(4584, 1147)

We have 4584 people in the train set, and 1147 people in the test set. Nice! Let's check the total number of images as well.

In [90]:
num_train_images = train_persons.sum()['num_photos']
num_test_images = test_persons.sum()['num_photos']

print(f'Number of training images: {num_train_images}, Number of testing images: {num_test_images}')

Number of training images: 9234, Number of testing images: 2242


That's a pretty nice split so far. The next thing we can do is incorporate the people with unique image counts into the train and test set. For this, we can just manually assign them.

#### c. Manually Assigning People with Unique Image Counts into Train or Test Set